# Compile a standard residual ambient spectrum
following [ANSI/ASA S3/SC1.100-2014 ANSI/ASA S12.100-2014 *Methods to Define and Measure the Residual Sound in Protected Natural and Quiet Residential Areas*](https://webstore.ansi.org/standards/asa/ansiasas3sc11002014s12) methods

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt, rcParams, rc
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle
import matplotlib.ticker as mticker
import scipy
from scipy import ndimage, stats, signal, fft
from sklearn import decomposition
from skimage.transform import resize
import ruptures as rpt
import os
import datetime as dt
import time
import sys

# general NSNSD acoustical tools
# =======================================================================================================
repo_dir = r"C:\Users\DBetchkal\CODE\GITHUB" # ADJUST TO YOUR LOCAL DIRECTORY
# =======================================================================================================
sys.path.append(os.path.join(repo_dir, "iyore"))
sys.path.append(os.path.join(repo_dir, "derivedDataFunctions"))
sys.path.append(os.path.join(repo_dir, "NPS-ActiveSpace"))

# # ========== IF YOU DON'T HAVE THE TOOLS ALREDY, USE =========================
# !git clone https://github.com/nationalparkservice/iyore.git
# !git clone https://github.com/dbetchkal/derivedDataFunctions.git
# !git clone https://github.com/dbetchkal/NPS-ActiveSpace.git

import iyore
from nps_active_space.utils.models import Nvspl
from derivedDataFunctions import *

def ambience_from_pickle(ambience_src: str) -> pd.Series:
    
    Lx = pd.read_pickle(ambience_src)
    return Lx

def spectrum_plot(amb_series, ylabel="Sound Level"):

    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    ax.plot(amb_series.index.astype('float'), amb_series, 
            ls=":", lw=0.5, marker="o", ms=4, color="k")
    ax.set_xscale('log')
    ax.set_xlabel("Frequency (Hz)", labelpad=20)
    ax.set_ylabel(ylabel, labelpad=20)
    return fig, ax


#### Project setup! <font color="green">please edit</font>
- `ds`
- `des`
- `x`

In [ ]:
# A dataset of Type-1 acoustic monitoring deployments
ds = iyore.Dataset(r"D:\Sound\MORU_ActiveSpace") # where is the `iyore` ".structure.txt" file located?

des = "MORU0022025" # a deployment designator, styled as UNITSITEYYYY or UNIT000YYYY
x = 90 # exceedance level in the form Lx

dataDirectory = [deployment.path for deployment in ds.dataDir(unit=des[:4], site=des[4:-4], deploy_year=des[-4:])][0]
dataDirectory # find the deployment you wish, note the index

### Compile the estimate and save

In [ ]:
nv = Nvspl([e.path for e in ds.nvspl(unit=des[:4], site=des[4:-4], year=des[-4:])]) # parse the acoustic record
nv_lowWind = nv.loc[nv['WindSpeed'] <= 5.0, :] # we are only concerned with viable observations (where wind speed ≤5.0 m/s)

# compute the spectral exceedance
# x = 50 # exceedance level in the form Lx
Lx = nv_lowWind.loc[:, "12.5":"20000"].quantile(1 - (x/100))
Lx.name = des + " " + "residual ambient spectrum"

print(Lx)

# save it to a pickle file
output_filename = os.path.join(dataDirectory, des+"_L"+str(x)+".pkl")
Lx.to_pickle(output_filename, compression='infer', protocol=2)

### Reload the estimate and plot

In [ ]:
amb = ambience_from_pickle(output_filename) # or paste in your own path

fig, ax = spectrum_plot(amb)
plt.show()
plt.close()